In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
class Interval:
    def __init__(self, inf, sup=None):
        if (not (isinstance(inf, float) or isinstance(inf, int)) or not (
                isinstance(sup, float) or isinstance(sup, int) or sup is None)):
            raise TypeError()
        if sup is None:
            sup = inf
        elif inf > sup:
            raise ValueError()
        self.inf, self.sup = float(inf), float(sup)

    def __abs__(self):
        args = (abs(self.inf), abs(self.sup))
        if 0.0 in self:
            return Interval(0.0, max(args))
        return Interval(min(args), max(args))

    def __add__(self, other):
        if type(self) is not type(other):
            other = Interval(other)
        return Interval(self.inf + other.inf, self.sup + other.sup)

    def __contains__(self, item) -> bool:
        return self.inf <= item and item <= self.sup

    def __eq__(self, other) -> bool:
        if type(self) is not type(other):
            other = Interval(other)
        return (self.inf, self.sup) == (other.inf, other.sup)

    def __ge__(self, other) -> bool:
        if type(self) is not type(other):
            other = Interval(other)
        return self.inf >= other.sup

    def __gt__(self, other) -> bool:
        if type(self) is not type(other):
            other = Interval(other)
        return self.inf > other.sup
    
    def __iter__(self):
        yield self.inf
        yield self.sup

    def __le__(self, other) -> bool:
        if type(self) is not type(other):
            other = Interval(other)
        return self.sup <= other.inf

    def __lt__(self, other) -> bool:
        if type(self) is not type(other):
            other = Interval(other)
        return self.sup < other.inf

    def __ne__(self, other) -> bool:
        return not self == other

    def __mul__(self, other):
        if type(self) is not type(other):
            other = Interval(other)
        args = (self.inf * other.inf, self.inf * other.sup,
            self.sup * other.inf, self.sup * other.sup)
        return Interval(min(args), max(args))

    def __neg__(self):
        return Interval(-self.sup, -self.inf)

    def __pos__(self):
        return Interval(self.inf, self.sup)

    def __repr__(self) -> str:
        return F'Interval({self.inf}, {self.sup})'

    def __str__(self) -> str:
        return F'[{self.inf}, {self.sup}]'

    def __sub__(self, other):
        if type(self) is not type(other):
            other = Interval(other)
        return Interval(self.inf - other.sup, self.sup - other.inf)

    def __truediv__(self, other):
        if type(self) is not type(other):
            other = Interval(other)
        if 0.0 in other:
            raise ZeroDivisionError()
        return self * Interval(1.0 / other.sup, 1.0 / other.inf)

In [ ]:
import numpy as np

def gaussian_elimination(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    if (not isinstance(a, np.ndarray) or not isinstance(b, np.ndarray)
            or a.dtype != b.dtype):
        raise TypeError()
    eps, n = np.finfo(float).eps, len(a)
    if n < 1 or a.shape != (n, n) or b.shape != (n, ):
        raise ValueError()
    a, b = np.copy(a), np.copy(b)
    for i in range(n):
        i_max = i + np.argmax(np.abs(a[i :, i]))
        abs_value = np.abs(a[i_max, i])
        if ((a.dtype == Interval and abs_value.inf < eps) or abs_value < eps):
            raise ZeroDivisionError()
        if i_max != i:
            first, second = [i_max, i], [i, i_max]
            a[first], b[first] = a[second], b[second]
        for index in range(n):
            if index == i:
                continue
            mu = -a[index, i] / a[i, i]
            a[index, i:] += a[i, i:] * mu
            b[index] += b[i] * mu
    return b / np.diagonal(a)

def equivalent_form(a: np.ndarray, b: np.ndarray) -> tuple:
    n = len(a)
    for i in range(n):
        div = a[i, i]
        for j in range(n):
            a[i, j] = a[i, j] * 0.0 if i == j else -a[i, j] / div
        b[i] /= div
    return a, b

def successive_iteration(a: np.ndarray, b: np.ndarray,
        max_iter: int = 10000) -> np.ndarray:
    if (not isinstance(a, np.ndarray) or not isinstance(b, np.ndarray)
            or a.dtype != b.dtype or not isinstance(max_iter, int)):
        raise TypeError()
    n = len(a)
    if n < 1 or a.shape != (n, n) or b.shape != (n, ) or max_iter < 1:
        raise ValueError()
    alpha, beta = equivalent_form(np.copy(a), np.copy(b))

    x_curr = x_prev = beta
    for _ in range(max_iter):
        x_curr, x_prev = x_prev, x_curr
        x_curr = beta + alpha @ x_prev
    return x_curr

In [ ]:
from numpy.linalg import solve

a, b = np.array([
    [13.0, -6.0],
    [3.0, 10.0]
]), np.array([-5.0, 6.0])
o = solve(a, b)
assert np.allclose(gaussian_elimination(a, b), o)

interval_vectorize, epsilon = np.vectorize(Interval), Interval(1.0, 1.1)
a_interval = interval_vectorize(a) * epsilon
b_interval = interval_vectorize(b) * epsilon

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(-10, 10, 2 ** 10 + 1)
ab = (b[0] - a[0, 0] * x) / a[0, 1]
cd = (b[1] - a[1, 0] * x) / a[1, 1]

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white')
subplot.plot(x, ab, 'green', lw=2, label='AB')
subplot.plot(x, cd, 'blue', lw=2, label='CD')
subplot.plot(o[0], o[1], 'Xr', label='AB∩CD')
plt.legend()
plt.show()

In [ ]:
def interval_line(a: np.ndarray, b: Interval,
        start: float = -10.0, stop: float = 10.0, n: int = 1025) -> np.ndarray:
    if np.abs(a[1]).inf < np.finfo(float).eps:
        raise ValueError()
    y_inf_sup = np.vectorize(lambda x: tuple(b / a[1] - a[0] / a[1] * x))
    x = np.linspace(start, stop, n)
    return (x, *y_inf_sup(x))

In [ ]:
result = gaussian_elimination(a_interval, b_interval)
assert o[0] in result[0] and o[1] in result[1]
print(o)
print(result)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white')
x, y_inf, y_sup = interval_line(a_interval[0], b_interval[0])
subplot.fill_between(x, y_inf, y_sup, color='green', label='AB')
x, y_inf, y_sup = interval_line(a_interval[1], b_interval[1])
subplot.fill_between(x, y_inf, y_sup, color='blue', label='CD')
plt.legend()
plt.show()

In [ ]:
result = successive_iteration(a_interval, b_interval)
assert o[0] in result[0] and o[1] in result[1]
print(o)
print(result)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white')
x, y_inf, y_sup = interval_line(a_interval[0], b_interval[0])
subplot.fill_between(x, y_inf, y_sup, color='green', label='AB')
x, y_inf, y_sup = interval_line(a_interval[1], b_interval[1])
subplot.fill_between(x, y_inf, y_sup, color='blue', label='CD')
plt.legend()
plt.show()